In [33]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
import tsfresh
from pmdarima.arima import ADFTest
from tsfresh import extract_features
from statsmodels.tsa.arima.model import ARIMA

In [34]:
values = read_csv("../../Time_series_data.csv")

In [35]:
values.head(13)

,Month,Sales
0,2013-01,2815
1,2013-02,2672
2,2013-03,2755
3,2013-04,2721
4,2013-05,2946
5,2013-06,3036
6,2013-07,2282
7,2013-08,2212
8,2013-09,2922
9,2013-10,4301


In [36]:

values = values.rename(columns={values.columns[1]: 'Data'})
values = values.rename(columns={values.columns[0]: 'Time'})
values['Time'] = pd.to_datetime(
    values['Time'], errors='coerce')

In [37]:
freq = pd.infer_freq(values["Time"])

In [38]:
# end_date = pd.to_datetime(x) + pd.DateOffset(months=10)
# print(end_date)

In [39]:
values.set_index('Time', inplace=True)

In [40]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    values, test_size=0.2, shuffle=False)

rnn_train = train
rnn_test = test

In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(rnn_train)
scaled_train = scaler.transform(rnn_train)
scaled_test = scaler.transform(rnn_test)

In [42]:
from keras.preprocessing.sequence import TimeseriesGenerator

# define generator
n_input = 12
n_features = 1
generator = TimeseriesGenerator(
    scaled_train, scaled_train, length=n_input, batch_size=1)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [43]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu',
            input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()
model.fit(generator, epochs=50)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               40800     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40901 (159.77 KB)
Trainable params: 40901 (159.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
72/72 [==============================] - 7s 11ms/step - loss: 0.0719
Epoch 2/50
72/72 [==============================] - 1s 12ms/step - loss: 0.0476
Epoch 3/50
72/72 [==============================] - 1s 11ms/step - loss: 0.0449
Epoch 4/50
72/72 [==============================] - 1s 16ms/step - loss: 0.0382
Epoch 5/50
72/72 [==============================] - 1s 14ms/step - loss: 0.0284
Epoch 6/50
7

In [44]:
last_train_batch = scaled_train[-12:]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))



In [45]:
last_train_batch

array([[[0.08636474],
        [0.10742931],
        [0.14558859],
        [0.1755651 ],
        [0.1096978 ],
        [0.19549542],
        [0.21421048],
        [0.0133679 ],
        [0.29555213],
        [0.39301628],
        [0.66993438],
        [0.93194523]]])

In [46]:
model.predict(last_train_batch)

test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

1/1 [==============================] - 1s 569ms/step


In [47]:
first_eval_batch

array([[0.08636474],
       [0.10742931],
       [0.14558859],
       [0.1755651 ],
       [0.1096978 ],
       [0.19549542],
       [0.21421048],
       [0.0133679 ],
       [0.29555213],
       [0.39301628],
       [0.66993438],
       [0.93194523]])

In [48]:
for i in range(len(rnn_test)):
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]

    # append the prediction into the array
    test_predictions.append(current_pred)
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:, 1:, :], [
                                [current_pred]], axis=1)
    print(current_batch)

1/1 [==============================] - 0s 60ms/step


[[[0.10742931]
  [0.14558859]
  [0.1755651 ]
  [0.1096978 ]
  [0.19549542]
  [0.21421048]
  [0.0133679 ]
  [0.29555213]
  [0.39301628]
  [0.66993438]
  [0.93194523]
  [0.15925267]]]
1/1 [==============================] - 0s 74ms/step
[[[0.14558859]
  [0.1755651 ]
  [0.1096978 ]
  [0.19549542]
  [0.21421048]
  [0.0133679 ]
  [0.29555213]
  [0.39301628]
  [0.66993438]
  [0.93194523]
  [0.15925267]
  [0.14729077]]]
1/1 [==============================] - 0s 75ms/step
[[[0.1755651 ]
  [0.1096978 ]
  [0.19549542]
  [0.21421048]
  [0.0133679 ]
  [0.29555213]
  [0.39301628]
  [0.66993438]
  [0.93194523]
  [0.15925267]
  [0.14729077]
  [0.15666816]]]
1/1 [==============================] - 0s 75ms/step
[[[0.1096978 ]
  [0.19549542]
  [0.21421048]
  [0.0133679 ]
  [0.29555213]
  [0.39301628]
  [0.66993438]
  [0.93194523]
  [0.15925267]
  [0.14729077]
  [0.15666816]
  [0.16758396]]]
1/1 [==============================] - 0s 60ms/step
[[[0.19549542]
  [0.21421048]
  [0.0133679 ]
  [0.29555213]
  [0

In [49]:
current_batch

array([[[0.45216081],
        [0.78129596],
        [1.0006789 ],
        [0.1963079 ],
        [0.16419023],
        [0.16006632],
        [0.16091104],
        [0.14702593],
        [0.16978379],
        [0.18239316],
        [0.14809808],
        [0.26552376]]])

In [50]:
true_predictions = scaler.inverse_transform(test_predictions)
rnn_test['Predictions'] = true_predictions

In [51]:
rnn_test

,Data,Predictions
Time,,
2020-01-01,3934,3538.655748
2020-02-01,3162,3391.009928
2020-03-01,4286,3506.755054
2020-04-01,4676,3641.488785
2020-05-01,5010,3273.577289
2020-06-01,4874,3808.749842
2020-07-01,4633,4005.276130
2020-08-01,1659,2970.067695
2020-09-01,5951,4983.251142
